In [1]:
import numpy as np
import random
from mnist import MNIST
import matplotlib.pyplot as plt

In [2]:
mndata = MNIST('../HW1/Dataset')
trainX = np.array(mndata.load_training()[0])[:50000]
trainY = np.array(mndata.load_training()[1])[:50000]

testX = np.array(mndata.load_testing()[0])[:1000]
testY = np.array(mndata.load_testing()[1])[:1000]

In [3]:
valIndices = np.random.choice(len(trainX), 2000)
nonValIndices = [x for x in range(len(trainX)) if x not in valIndices]

valX = trainX[valIndices]
valY = trainY[valIndices]

trainX = trainX[nonValIndices]
trainY = trainY[nonValIndices]

testX = np.array(mndata.load_testing()[0])[:2000]
testY = np.array(mndata.load_testing()[1])[:2000]

def feat(data,i):
    return data[i].tolist()

def oneHot(clas, noOfClasses):
    feat = np.zeros(noOfClasses)
    feat[clas] = 1;
    return feat

trnX = np.array([feat(trainX,i) for i in range(trainX.shape[0])])/256.0
trnY = np.array([oneHot(trainY[i], 10) for i in range(trainX.shape[0])])

tstX = np.array([feat(testX,i) for i in range(testX.shape[0])])/256.0
tstY = np.array([oneHot(testY[i], 10) for i in range(testX.shape[0])])

valX = np.array([feat(valX,i) for i in range(valX.shape[0])])/256.0
valY = np.array([oneHot(valY[i], 10) for i in range(valX.shape[0])])

In [4]:
trnY[0:10]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [21]:
def sigmoid(x):
    return 1.0/(1+np.exp(-1*np.array(x)))

def grad(x):
    return x*(1-x)

def softmax(x):
    x = np.exp(x)
    x = x/x.sum(axis=1)[:, None]
    return x
    
maxValAcc = 0
testAcc = 0
lr = 0.0001

# randomly initialize our weights with mean 0
n_hid_1 = 50
n_hid_2 = 80
W1 = 2*np.random.random((784,n_hid_1)) - 1
bias1 = 2*np.random.random((n_hid_1)) - 1
W2 = 2*np.random.random((n_hid_1,n_hid_2)) - 1
bias2 = 2*np.random.random((n_hid_2)) - 1
W3 = 2*np.random.random((n_hid_2,10)) - 1
bias3 = 2*np.random.random((10)) - 1

for j in xrange(100):
    A1 = np.dot(trnX, W1) + bias1
    l1 = sigmoid(A1)
    
    A2 = np.dot(l1, W2) + bias2
    l2 = sigmoid(A2)
    
    A3 = np.dot(l2, W3) + bias3
    l3 = softmax(A3)

    # Errors in output layer
    d3 = (l3 - trnY)
    dbias3 = np.sum(d3, axis = 0)
        
    # Delta of W3
    dW3 = np.dot(l2.T, d3)

    # Errors in 2nd hidden layer
    d2 = np.dot(d3, W3.T)*grad(l2)
    dbias2 = np.sum(d2, axis = 0)    
    
    # Delta W2
    dW2 = np.dot(l1.T, d2)
    
    # Errors in 2nd hidden layer
    d1 = np.dot(d2, W2.T)*grad(l1)
    dbias1 = np.sum(d1, axis = 0)    
    
    # Delta W3
    dW1 = np.dot(trnX.T, d1)
    
    W3 -= lr*dW3
    bias3 -= lr*dbias3
    W2 -= lr*dW2
    bias2 -= lr*dbias2
    W1 -= lr*dW1
    bias1 -= lr*dbias1
    
    prediction = softmax(np.dot(sigmoid(np.dot(sigmoid(np.dot(valX, W1)+bias1), W2)+bias2), W3)+bias3)
    correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(valY, axis = 1), np.argmax(prediction, axis = 1))]
    valAcc = np.sum(correct)*100.0/len(valX)
    if(valAcc > maxValAcc):
        prediction = softmax(np.dot(sigmoid(np.dot(sigmoid(np.dot(tstX, W1)+bias1), W2)+bias2), W3)+bias3)
        correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(tstY, axis = 1), np.argmax(prediction, axis = 1))]
        testAcc = np.sum(correct)*100.0/len(tstX)
        print testAcc

14.75
13.4
15.45
10.95
9.9
20.55
21.3
25.85
29.9
36.4
37.6
39.5
34.35
30.4
14.95
29.6
48.25
49.45
51.4
51.0
47.55
43.0
48.1
57.1
62.2
63.15
61.4
58.6
54.1
60.1
62.3
71.1
69.65
72.8
71.35
73.1
70.0
67.1
59.0
53.3
57.45
61.3
68.15
67.65
68.4
69.15
68.85
71.4
69.3
72.65
71.15
74.25
72.5
75.1
73.45
76.15
74.25
77.0
75.25
77.5
75.8
78.3
76.65
78.75
77.55
79.1
78.05
79.85
79.15
79.95
79.65
80.6
80.15
81.5
80.85
81.8
82.05
82.4
82.5
82.9
82.85
82.9
83.65
83.3
84.05
83.55
84.4
83.65
84.75
83.65
84.95
83.85
84.95
84.1
85.1
84.1
85.35
84.15
85.45
84.25
